## Overview
We found that some paths after snap-to-grid are obviously not right.

Such as the image in the left. 

What I did in this notebook is, if the path obviously cross a wall, fix the path like the following image. 

<img src= "https://i.imgur.com/a8B34Hm.png" alt ="the structure of the model" style='width: 500px;'>

To achieve it, I thought I need to calculate the distance of between 2 points take the wall into account, not the direct distance.

So I created the character version of maps. Then I applied Breadth First Search to get the rough distance in the notebook. 

In [ ]:
import pandas as pd
import numpy as np
import glob
from scipy.spatial.distance import cdist
import json
import matplotlib.pylab as plt
import pickle

from shapely.geometry import Point, LineString
from tqdm import tqdm
import warnings
from shapely.geometry import Polygon
from shapely.ops import nearest_points
import multiprocessing
import os
import math

import multiprocessing
from multiprocessing import Pool

num_cores = multiprocessing.cpu_count()

warnings.simplefilter('ignore')

In [ ]:
# options

train_waypoins = pd.read_csv('../input/indoor-location-train-waypoints/train_waypoints.csv')
GRID_SIZE = 1 # meters
WALL_THRESHOLD = 0.5 

In [ ]:
submission = pd.read_csv('../input/my-preds/submission_3.645.csv')
target_sites = submission['site_path_timestamp'].apply(lambda x: x.split('_')[0]).unique()

In [ ]:
metadata = pd.DataFrame(glob.glob('../input/indoor-location-navigation/metadata/*'))
metadata.columns=['path']
metadata['site'] = metadata['path'].apply(lambda x: x.split('/')[-1])
metadata = metadata[metadata['site'].isin(target_sites)]['path'].values

In [ ]:
floor_map = {"B2":-2, "B1":-1, "F1":0, "F2": 1, "F3":2,
             "F4":3, "F5":4, "F6":5, "F7":6,"F8":7,"F9":8,
             "1F":0, "2F":1, "3F":2, "4F":3, "5F":4, "6F":5,
             "7F":6, "8F": 7, "9F":8}

In [ ]:
GRID_AREA = GRID_SIZE * GRID_SIZE

def generate_array_map(site):
    
    print(f' Started {site}')
    
    metadata_floor_paths = glob.glob(f'../input/indoor-location-navigation/metadata/{site}/*')
    
    for metadata_floor_path in metadata_floor_paths:
        floor = metadata_floor_path.split('/')[-1]
        floor_val = floor_map[floor]

        corridor_path = f'../input/indoor-location-navigation-scaled-geojson/scaled_geojson/{site}/{floor}/shapely_geometry.pkl'
        with open(corridor_path, 'rb') as f:
            corridor = pickle.load(f)
                        
        minx, miny, maxx, maxy = corridor.bounds

        encoded_map = []
        for y in range(0, math.ceil(maxy)):
            encoded_map.append([])
            for x in range(0, math.ceil(maxx)):

                cell_polygon = Polygon([(x * GRID_SIZE, y * GRID_SIZE),
                                        (x * GRID_SIZE, (y + 1) * GRID_SIZE),
                                        ((x + 1) * GRID_SIZE, (y + 1) * GRID_SIZE),
                                        ((x + 1) * GRID_SIZE, y * GRID_SIZE)])

                intersection_area = GRID_AREA - corridor.intersection(cell_polygon).area
                if intersection_area >= WALL_THRESHOLD:
                    encoded_map[y].append('#')
                    continue


                encoded_map[y].append('.')

        waypoints_on_floor = train_waypoins[(train_waypoins['site']==site) & (train_waypoins['floor']==floor_val)]
        for i, wp in waypoints_on_floor.iterrows():
            x = math.floor(wp['x'] / GRID_SIZE)
            y = math.floor(wp['y'] / GRID_SIZE)

            encoded_map[y][x] = '*'
            
        encoded_map = np.array(encoded_map).T
        
        with open(f'{site}_{floor}_array_map.pkl', 'wb') as f:
            pickle.dump(encoded_map, f)

    print(f' Done! {site}')

In [ ]:
with Pool(num_cores) as pool:
    pool.map(generate_array_map, [t for t in target_sites])  

In [ ]:
def display_aa_map(array_map):
    ar_map = array_map
    for line in reversed(ar_map):
        print()
        for cell in line:
            print(cell, end='')

In [ ]:
# example   
with open('./5da138274db8ce0c98bbd3d2_F2_array_map.pkl', 'rb') as f:
    ex= pickle.load(f)
    
display_aa_map(np.array(ex))